### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [15]:
import os
import json
import datetime
import subprocess
import platform

In [16]:
def log2(result, name, action):
    if result.returncode != 0:
        print("👎🏻 ", name, " was NOT ", action, ": ", result.stderr)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

def run_command(cmd_str, name, action):
    oss=platform.system()
    if (oss=='Windows'):
        cmd=cmd_str.split(' ')
    else:
        cmd=cmd_str
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, shell=True, universal_newlines=True)

        #stdout = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        log2(result, name, action)
    except subprocess.CalledProcessError as e:
        print(f"Unknown Error\n{e.stderr}\n")
        raise e
    return result

In [18]:
lab_prefix="aif-pevo2-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

#cmd_str= f"az deployment group show --name {deployment_name} -g {resource_group_name} -o json"
#result=run_command(cmd_str, deployment_name, "deployment group show")
#cmd_rtc = result.returncode
#cmd_out = result.stdout
#cmd_err = result.stderr


#if ( (oss=='Windows' and exit_code != 1) or (oss=='Linux' and exit_code != 0) ): 
#if ( cmd_rtc != 0 ): 
#    print("Deployment Error Exiting", cmd_rtc, deployment_name, resource_group_name)
#    raise KeyboardInterrupt

#deployment = json.loads(cmd_out)

        #raise SystemExit("Stopping the execution of the cell.") 

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json   
#print(deployment_stdout)
if deployment_stdout.n.startswith("ERROR"):
    print("👎🏻 No active deployments: \n", deployment_stdout)
    raise KeyboardInterrupt

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

deployment = json.loads(deployment_stdout.n)

for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")  
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
    except:
        print("✌🏻 ", resource_id, " ignored")
delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")



👎🏻 No active deployments:  ["ERROR: (ResourceGroupNotFound) Resource group 'aif-pevo2-function-calling' could not be found.", 'Code: ResourceGroupNotFound', "Message: Resource group 'aif-pevo2-function-calling' could not be found."]


KeyboardInterrupt: 

: 